# Stock purchase recommendations with Machine Learning - Model Training

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook # progress bar
import fastparquet
import pickle

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [2]:
# pd.set_option('display.max_columns', 1500)

## 1) Load Training and Test Data

In [3]:
# load the training and test datae from feature engineering step:
X_train = fastparquet.ParquetFile('../data/processed/X_train.parq').to_pandas()
X_test = fastparquet.ParquetFile('../data/processed/X_test.parq').to_pandas()
y_train = pickle.load(open('../data/processed/y_train.pkl', 'rb'))
y_test = pickle.load(open('../data/processed/y_test.pkl', 'rb'))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7024, 693), (1756, 693), (7024,), (1756,))

In [4]:
# quick inspection
X_train.tail()

,AdjVolume-0,AdjVolume-1,AdjVolume-2,AdjVolume-3,AdjVolume-4,AdjVolume-5,AdjVolume-6,AdjVolume-7,AdjVolume-8,AdjVolume-9,...,weekday,day,AAPL.US,ABBV.US,AMZN.US,CSCO.US,GE.US,INTC.US,MSFT.US,NFLX.US
Date,,,,,,,,,,,,,,,,,,,,,
2017-05-10,1.0,0.855512,0.860082,0.885949,0.822510,0.968459,0.814462,0.889968,1.065802,2.269711,...,2,10,0,1,0,0,0,0,0,0
2017-05-10,1.0,1.066204,0.762742,0.850528,0.791760,0.827486,0.821882,0.769241,1.147349,1.519654,...,2,10,0,0,0,1,0,0,0,0
2017-05-10,1.0,1.400148,1.883068,1.043509,0.906711,1.758551,1.548577,1.278464,0.788735,0.543387,...,2,10,1,0,0,0,0,0,0,0
2017-05-10,1.0,1.550622,1.617839,1.371090,1.142760,1.701144,1.824343,2.599446,3.493653,2.042627,...,2,10,0,0,1,0,0,0,0,0
2017-05-10,1.0,1.250876,1.033853,1.058335,1.200615,1.609998,1.289137,1.754546,2.182534,1.805805,...,2,10,0,0,0,0,0,0,1,0


In [5]:
X_test.head()

,AdjVolume-0,AdjVolume-1,AdjVolume-2,AdjVolume-3,AdjVolume-4,AdjVolume-5,AdjVolume-6,AdjVolume-7,AdjVolume-8,AdjVolume-9,...,weekday,day,AAPL.US,ABBV.US,AMZN.US,CSCO.US,GE.US,INTC.US,MSFT.US,NFLX.US
Date,,,,,,,,,,,,,,,,,,,,,
2017-05-11,1.0,0.637548,0.797493,0.659131,0.674739,0.765449,1.026450,0.821886,1.118606,1.391469,...,3,11,0,0,0,0,0,0,1,0
2017-05-11,1.0,1.106919,0.822218,0.821698,0.847892,0.767362,1.013889,1.647503,1.138211,2.605818,...,3,11,0,0,0,0,0,1,0,0
2017-05-11,1.0,1.234871,0.702247,0.863877,0.960571,0.965947,0.698233,0.900488,1.619883,1.140485,...,3,11,0,0,0,0,0,0,0,1
2017-05-11,1.0,1.050633,1.120189,0.801362,0.893593,0.831849,0.869383,0.863496,0.808190,1.205442,...,3,11,0,0,0,1,0,0,0,0
2017-05-11,1.0,1.002882,1.404183,1.888495,1.046517,0.909324,1.763619,1.553040,1.282149,0.791008,...,3,11,1,0,0,0,0,0,0,0


In [6]:
y_train[:5]

Date
2013-11-13    False
2013-11-13    False
2013-11-13    False
2013-11-13     True
2013-11-13     True
Name: setup_for_profitable_trade, dtype: bool

## 2) Build Initial Model - RandomForestClassifier

Start the ML process with a simple out of the box RandomForestClassifier to get a base line and validate that the training data is functioning with sklearn and can generate some predictions.

In [7]:
# define a simple pipeline
pipeline = Pipeline([
    ('standardScaler', StandardScaler()),
    ('randomForest', RandomForestClassifier())
])

# inspect parameters
pipeline.get_params()

{'memory': None,
 'randomForest': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'randomForest__bootstrap': True,
 'randomForest__class_weight': None,
 'randomForest__criterion': 'gini',
 'randomForest__max_depth': None,
 'randomForest__max_features': 'auto',
 'randomForest__max_leaf_nodes': None,
 'randomForest__min_impurity_decrease': 0.0,
 'randomForest__min_impurity_split': None,
 'randomForest__min_samples_leaf': 1,
 'randomForest__min_samples_split': 2,
 'randomForest__min_weight_fraction_leaf': 0.0,
 'randomForest__n_estimators': 'warn',
 'randomForest__n_jobs': None,
 'randomForest__oob_score': F

In [8]:
# fit the pipeline with all default parameters
pipeline.fit(X_train, y_train.reset_index().setup_for_profitable_trade)

C:\Users\florian.lindstaedt\AppData\Local\Continuum\anaconda3\envs\trader_env2\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\florian.lindstaedt\AppData\Local\Continuum\anaconda3\envs\trader_env2\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\florian.lindstaedt\AppData\Local\Continuum\anaconda3\envs\trader_env2\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('standardScaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [9]:
# create prediction
y_pred_firstRF = pipeline.predict(X_test)

# save for backtesting in separate notebook
pickle.dump(y_pred_firstRF, open('../data/model_predictions/y_pred_firstRF.pkl', 'wb'))

C:\Users\florian.lindstaedt\AppData\Local\Continuum\anaconda3\envs\trader_env2\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


We now have a first prediction on the dataset - let's look into the performance of the default settings. To understand the result, we will look at a few different metrics: the accuracy, precision, recall, f1-score, and the confusion matrix.

It is important to note that our data is imbalanced (not the same number of days with trade signal as with no signal). Therefore, F scores are a better metric than ROC-AUC or accuracy. We can not easily fix the imbalance since we are looking at time series data and resampling might introduce lookahead bias.

The ultimate test of the quality of the prediction is to backtest the results (ie simulate financial performance based on the predictions). This will be done in the next workbook. For now let's inspect some of the basic metrics.

In [10]:
print(classification_report(y_test, y_pred_firstRF))

              precision    recall  f1-score   support

       False       0.64      0.81      0.72      1153
        True       0.27      0.14      0.18       603

   micro avg       0.58      0.58      0.58      1756
   macro avg       0.46      0.47      0.45      1756
weighted avg       0.52      0.58      0.53      1756



In [11]:
accuracy_score(y_test, y_pred_firstRF)

0.5780182232346242

In [12]:
confusion_matrix(y_test, y_pred_firstRF, labels=[False, True])

array([[931, 222],
       [519,  84]], dtype=int64)

The confusion matrix indicates a large number of False Positives (bad because if we used those to trade, we would enter a trade that turns out not to be as profitable as desired) and False Negatives (bad because they mean missed opportunities for us to enter profitable trades).

The good news is that our out-of-the-box model was able to predict some True Positives but the results in terms of financial performance are most likely very bad (we saved off the prediction and will look at this later).

Even though the model seems to perform poorly, we might be able to learn something from it:

Let's look into the relative feature importances to see if anything stands out

In [13]:
def print_feature_importances(estimator):

    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print(str(f + 1) + " importance: " + str(importances[indices[f]]) + ". feature name: " + X_train.columns[indices[f]])

In [14]:
      print_feature_importances(pipeline.named_steps['randomForest'])

Feature ranking:
1 importance: 0.007843464015185191. feature name: AdjCloseSMA50_chg-15
2 importance: 0.00716759599702165. feature name: AdjCloseSMA50-16
3 importance: 0.006612960999743579. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio-12
4 importance: 0.006414316728199856. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio-19
5 importance: 0.005767379995719915. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio-8
6 importance: 0.005162072700455432. feature name: AdjCloseSMA50_to_AdjClose_ratio-19
7 importance: 0.004816379481689198. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio-2
8 importance: 0.004700652083304829. feature name: AdjCloseSMA50_chg-10
9 importance: 0.00470030541750586. feature name: AdjCloseSMA50_to_AdjClose_ratio-4
10 importance: 0.004639709665878761. feature name: AdjCloseSMA200_to_AdjClose_ratio-10
11 importance: 0.004598620694147068. feature name: AdjCloseSMA50_chg-19
12 importance: 0.004593391296119925. feature name: AdjCloseSMA50_to_AdjCloseSMA200_rati

At the very bottom of the list are the RSI_above_80 and RSI_below_20 indicators. RSI is an indicator of extremes so having these features not play an important role means that the model did not notice many extremes and having predictive power.
The top section contains many of the SMA50 to SMA200 ratios. The SMA 50 and 200 crossovers or distances are classic momentum indicators (SMA 50 over SMA 200 = uptrend, below = downtrend).
Since the time period in the training data is that of a general market uptrend, it sounds intuitively reasonable that momentum indicators are important.

## 3) Improve RandomForest model with GridSearch

Given the imbalance of our data and the desire to avoid False Positives, we will use a modified F1 score that places stronger emphasis on precision than on recall (they have the same weight in the F1 score). We will skew it towards precision by setting beta to 0.5 and towards recall bysetting beta to 2.0. The combinations were used in separate runs of GridSearchCV and results were saved.

In [15]:
# place higher focus on precision (ie getting most TP and minimize FP) than on recall (ie minimize FN)
# since placing a trade that was based on a FP will be costly and hurts more than missing a trade due to a FN
fhalf_scorer = make_scorer(fbeta_score, beta=0.5) # low beta favors precision over recall
ftwo_scorer = make_scorer(fbeta_score, beta=2) # high beta favors recall over precision

For a GridSearch, we will define a wide range of parameters to be used in the pipeline. The new fhalf_scorer will be used for optimization.

In [16]:
parameters = {
    'randomForest__min_samples_leaf': [2, 5, 10],
    'randomForest__n_estimators' : [10, 20, 50, 100],
    'randomForest__max_features': [5, 'sqrt', 'log2'], # log2 of 690 = 9, sqrt of 690 = 20
    'randomForest__max_depth' : [4, 5, 6, 7, 8],
    'randomForest__criterion' :['gini', 'entropy']
}

my_cv = TimeSeriesSplit(n_splits=3)
cv = GridSearchCV(pipeline, param_grid=parameters, cv=my_cv, scoring='f1', n_jobs=-1, verbose=10) # uses fhalf_scorer
cv.fit(X_train, y_train)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   44.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   49.5s
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\FLORIA~1.LIN\\AppData\\Local\\Temp\\joblib_memmapping_folder_21776_1440040354\\21776-1454848104320-ce2dae511bb24195bc3e7fc4ed44e2a1.pkl'

In [ ]:
print(cv.best_params_)
pickle.dump(cv.best_params_, open('../models/GridSearch_vbetaone.pkl', 'wb'))

In [ ]:
print_feature_importances(cv.best_estimator_.named_steps['randomForest'])

In [ ]:
y_pred_GridSearch = cv.predict(X_test)

# save for backtesting in separate notebook
pickle.dump(y_pred_GridSearch, open('../data/model_predictions/y_pred_GridSearch_vbetaone.pkl', 'wb'))

In [ ]:
print(classification_report(y_test, y_pred_GridSearch))

In [ ]:
accuracy_score(y_test, y_pred_GridSearch)

In [ ]:
confusion_matrix(y_test, y_pred_GridSearch, labels=[False, True])

## next: resume in backtesting notebook to understand and measure true performance of the model predictions